# Advanced Loan Data Visualizations

This notebook focuses on creating advanced visualizations to uncover patterns and trends in the loan data.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 7)

In [ ]:
# Load data
df = pd.read_csv('../data/loan_data.csv')

# Convert date columns
df['application_date'] = pd.to_datetime(df['application_date'])
df['approval_date'] = pd.to_datetime(df['approval_date'])
df['disbursement_date'] = pd.to_datetime(df['disbursement_date'])

# Add calculated fields
df['loan_to_income'] = df['loan_amount'] / df['annual_income']
df['total_interest'] = (df['monthly_payment'] * df['loan_term']) - df['loan_amount']

print("Data loaded successfully!")
print(f"Shape: {df.shape}")

## 1. Time Series Analysis - Loan Applications Over Time

In [ ]:
# Time series of loan applications
df['year_month'] = df['application_date'].dt.to_period('M')
monthly_loans = df.groupby('year_month').agg({
    'loan_id': 'count',
    'loan_amount': 'sum'
}).rename(columns={'loan_id': 'count', 'loan_amount': 'total_volume'})

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Loan count over time
axes[0].plot(monthly_loans.index.astype(str), monthly_loans['count'], 
             marker='o', linewidth=2, color='steelblue')
axes[0].set_title('Monthly Loan Applications', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Month')
axes[0].set_ylabel('Number of Loans')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(True, alpha=0.3)

# Loan volume over time
axes[1].plot(monthly_loans.index.astype(str), monthly_loans['total_volume'], 
             marker='s', linewidth=2, color='green')
axes[1].set_title('Monthly Loan Volume', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Month')
axes[1].set_ylabel('Total Loan Amount ($)')
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Risk Segmentation Dashboard

In [ ]:
# Create risk categories
df['risk_category'] = 'Low Risk'
df.loc[df['credit_score'] < 650, 'risk_category'] = 'Medium Risk'
df.loc[(df['credit_score'] < 650) | (df['debt_to_income'] > 0.5), 'risk_category'] = 'High Risk'

# Risk segmentation analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Risk distribution
risk_counts = df['risk_category'].value_counts()
axes[0, 0].pie(risk_counts.values, labels=risk_counts.index, autopct='%1.1f%%', 
               colors=['lightgreen', 'yellow', 'lightcoral'], startangle=90)
axes[0, 0].set_title('Loan Portfolio Risk Distribution', fontsize=12, fontweight='bold')

# Risk vs Loan Status
risk_status = pd.crosstab(df['risk_category'], df['loan_status'])
risk_status.plot(kind='bar', stacked=True, ax=axes[0, 1], colormap='tab10')
axes[0, 1].set_title('Loan Status by Risk Category', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Risk Category')
axes[0, 1].set_ylabel('Count')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].legend(title='Loan Status', bbox_to_anchor=(1.05, 1))

# Risk vs Average Interest Rate
risk_rate = df.groupby('risk_category')['interest_rate'].mean().sort_values()
axes[1, 0].barh(risk_rate.index, risk_rate.values, color=['lightgreen', 'yellow', 'lightcoral'])
axes[1, 0].set_title('Average Interest Rate by Risk Category', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Average Interest Rate (%)')
axes[1, 0].set_ylabel('Risk Category')

# Risk vs Average Loan Amount
risk_amount = df.groupby('risk_category')['loan_amount'].mean().sort_values()
axes[1, 1].barh(risk_amount.index, risk_amount.values, color=['lightgreen', 'yellow', 'lightcoral'])
axes[1, 1].set_title('Average Loan Amount by Risk Category', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Average Loan Amount ($)')
axes[1, 1].set_ylabel('Risk Category')

plt.tight_layout()
plt.show()

## 3. Performance Metrics by Loan Purpose

In [ ]:
# Purpose-based analysis
purpose_stats = df.groupby('purpose').agg({
    'loan_id': 'count',
    'loan_amount': ['mean', 'sum'],
    'interest_rate': 'mean',
    'credit_score': 'mean'
}).round(2)

purpose_stats.columns = ['count', 'avg_amount', 'total_volume', 'avg_rate', 'avg_score']
purpose_stats = purpose_stats.sort_values('total_volume', ascending=False)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Total volume by purpose
axes[0, 0].barh(purpose_stats.index, purpose_stats['total_volume'], color='teal', alpha=0.7)
axes[0, 0].set_title('Total Loan Volume by Purpose', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Total Volume ($)')
axes[0, 0].set_ylabel('Loan Purpose')

# Loan count by purpose
axes[0, 1].barh(purpose_stats.index, purpose_stats['count'], color='navy', alpha=0.7)
axes[0, 1].set_title('Loan Count by Purpose', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Number of Loans')
axes[0, 1].set_ylabel('Loan Purpose')

# Average interest rate by purpose
purpose_rate = purpose_stats.sort_values('avg_rate', ascending=True)
axes[1, 0].barh(purpose_rate.index, purpose_rate['avg_rate'], color='red', alpha=0.7)
axes[1, 0].set_title('Average Interest Rate by Purpose', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Average Interest Rate (%)')
axes[1, 0].set_ylabel('Loan Purpose')

# Average credit score by purpose
purpose_score = purpose_stats.sort_values('avg_score', ascending=True)
axes[1, 1].barh(purpose_score.index, purpose_score['avg_score'], color='purple', alpha=0.7)
axes[1, 1].set_title('Average Credit Score by Purpose', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Average Credit Score')
axes[1, 1].set_ylabel('Loan Purpose')

plt.tight_layout()
plt.show()

## 4. Credit Score Impact on Loan Terms

In [ ]:
# Credit score bands
df['credit_band'] = pd.cut(df['credit_score'], 
                            bins=[0, 600, 650, 700, 750, 850],
                            labels=['<600', '600-649', '650-699', '700-749', '750+'])

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Interest rate by credit band
sns.violinplot(data=df, x='credit_band', y='interest_rate', ax=axes[0, 0], palette='muted')
axes[0, 0].set_title('Interest Rate Distribution by Credit Score Band', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Credit Score Band')
axes[0, 0].set_ylabel('Interest Rate (%)')

# Loan amount by credit band
sns.boxplot(data=df, x='credit_band', y='loan_amount', ax=axes[0, 1], palette='Set2')
axes[0, 1].set_title('Loan Amount Distribution by Credit Score Band', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Credit Score Band')
axes[0, 1].set_ylabel('Loan Amount ($)')

# DTI by credit band
sns.boxplot(data=df, x='credit_band', y='debt_to_income', ax=axes[1, 0], palette='Set3')
axes[1, 0].set_title('Debt-to-Income Ratio by Credit Score Band', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Credit Score Band')
axes[1, 0].set_ylabel('DTI Ratio')

# Loan status by credit band
credit_status = pd.crosstab(df['credit_band'], df['loan_status'], normalize='index') * 100
credit_status.plot(kind='bar', stacked=True, ax=axes[1, 1], colormap='tab20')
axes[1, 1].set_title('Loan Status Distribution by Credit Score Band', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Credit Score Band')
axes[1, 1].set_ylabel('Percentage (%)')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].legend(title='Status', bbox_to_anchor=(1.05, 1))

plt.tight_layout()
plt.show()

## 5. Profitability Analysis

In [ ]:
# Calculate profitability metrics
df['estimated_loss'] = 0
df.loc[df['loan_status'] == 'Charged Off', 'estimated_loss'] = df['loan_amount'] * 0.8
df.loc[df['loan_status'].str.contains('Late', na=False), 'estimated_loss'] = df['loan_amount'] * 0.2
df['net_revenue'] = df['total_interest'] - df['estimated_loss']

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Revenue by loan status
revenue_by_status = df.groupby('loan_status')['net_revenue'].sum().sort_values(ascending=True)
colors = ['red' if x < 0 else 'green' for x in revenue_by_status.values]
axes[0, 0].barh(revenue_by_status.index, revenue_by_status.values, color=colors, alpha=0.7)
axes[0, 0].set_title('Net Revenue by Loan Status', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Net Revenue ($)')
axes[0, 0].set_ylabel('Loan Status')
axes[0, 0].axvline(x=0, color='black', linestyle='--', linewidth=1)

# Interest revenue vs losses
total_interest = df['total_interest'].sum()
total_losses = df['estimated_loss'].sum()
net_profit = total_interest - total_losses

categories = ['Interest\nRevenue', 'Estimated\nLosses', 'Net\nProfit']
values = [total_interest, -total_losses, net_profit]
colors_profit = ['green', 'red', 'blue']
axes[0, 1].bar(categories, values, color=colors_profit, alpha=0.7)
axes[0, 1].set_title('Overall Profitability Analysis', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Amount ($)')
axes[0, 1].axhline(y=0, color='black', linestyle='--', linewidth=1)

# Revenue by purpose
revenue_by_purpose = df.groupby('purpose')['net_revenue'].sum().sort_values(ascending=True)
colors_purpose = ['red' if x < 0 else 'green' for x in revenue_by_purpose.values]
axes[1, 0].barh(revenue_by_purpose.index, revenue_by_purpose.values, color=colors_purpose, alpha=0.7)
axes[1, 0].set_title('Net Revenue by Loan Purpose', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Net Revenue ($)')
axes[1, 0].set_ylabel('Loan Purpose')
axes[1, 0].axvline(x=0, color='black', linestyle='--', linewidth=1)

# Default rate analysis
status_counts = df['loan_status'].value_counts()
default_statuses = ['Charged Off', 'Late (31-120 days)', 'Late (16-30 days)']
default_count = sum([status_counts.get(s, 0) for s in default_statuses])
performing_count = len(df) - default_count

axes[1, 1].pie([performing_count, default_count], 
               labels=['Performing', 'At Risk/Default'],
               colors=['lightgreen', 'lightcoral'],
               autopct='%1.1f%%', startangle=90)
axes[1, 1].set_title('Portfolio Performance Status', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\nProfitability Summary:")
print(f"Total Interest Revenue: ${total_interest:,.2f}")
print(f"Total Estimated Losses: ${total_losses:,.2f}")
print(f"Net Profit: ${net_profit:,.2f}")
print(f"Default Rate: {(default_count/len(df)*100):.2f}%")